In [ ]:
#@title <-- Tap this if you play on Mobile { display-mode: "form" }
%%html
<b>Press play on the music player to keep the tab alive, then start KoboldCpp below</b><br/>
<audio autoplay="" src="https://raw.githubusercontent.com/KoboldAI/KoboldAI-Client/main/colab/silence.m4a" loop controls>

In [ ]:
# @title Load utils

def get_renamed_file(MODEL_FILE_URL):
    file = MODEL_FILE_URL
    return file.split(file[file.rfind("/")])[-1]
    

def get_target_repo(MODEL_ID):
    return f"https://huggingface.co/{MODEL_ID}/tree/main?not-for-all-audiences=true"


def get_target_links(MODEL_TYPE, MODEL_FILES, links):

    LINKS = []

    if MODEL_TYPE == "gguf":
        truth_links = MODEL_FILES if len(MODEL_FILES) > 0 else links
        LINKS = [link.replace("/blob/", "/resolve/") for link in truth_links if link.endswith(".gguf") and "/blob/" in link]
        return LINKS


def create_download_file(MODEL_TYPE, LINKS, shopping_list, renamed_file):
    with open(shopping_list, "a") as file:
        if MODEL_TYPE == "gguf":
            for url in LINKS:
                file.write(f"{url}\n out={renamed_file}\n")
        else:
            for url in LINKS:
                if url.endswith(".safetensors"):
                    file.write(f"{url}\n out={renamed_file}\n")
                elif url.endswith(".model"):
                    file.write(f"{url}\n out=tokenizer.model\n")
                else:
                    file.write(f"{url}\n")

In [ ]:
#@Install Backend & Deps
!apt update
!apt install aria2 -y

!echo Downloading KoboldCpp, please wait...
!wget -O dlfile.tmp https://kcpplinux.concedo.workers.dev && mv dlfile.tmp koboldcpp_linux
!test -f koboldcpp_linux && echo Download Successful || echo Download Failed
!chmod +x ./koboldcpp_linux

In [ ]:
OPTIONS = {
    1: {
        "id": "mradermacher/Fimbulvetr-11B-v2-GGUF",
        "file": "https://huggingface.co/mradermacher/Fimbulvetr-11B-v2-GGUF/blob/main/Fimbulvetr-11B-v2.Q5_K_S.gguf"
    }
}

PRESET = "1" #@param [1]

MODEL = OPTIONS[int(PRESET)]["file"].replace("/blob/", "/resolve/")

!aria2c -x 10 -o model.gguf --summary-interval=5 --download-result=default --allow-overwrite=true --file-allocation=none $MODEL

In [ ]:
#@LAUNCH!

!./koboldcpp_linux model.gguf --usecublas 0 mmq --multiuser --gpulayers 60 --contextsize 4096 --quiet --remotetunnel